In [1]:
!pip install weaviate-client

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [ ]:
import weaviate
import json

client = weaviate.Client(
    url = "https://my-weaviate-jc375ien.weaviate.network",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key="OGqz8B0C"),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-HuggingFace-Api-Key": "hf_D"  # Replace with your inference API key
    }
)

In [16]:
schema = client.schema.get()
print(schema)

{'classes': [{'class': 'Question', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2', 'options': {'waitForModel': True}, 'vectorizeClassName': True}}, 'multiTenancyConfig': {'enabled': False}, 'properties': [{'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Tue Aug  8 08:34:56 2023", 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-huggingface': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'question', 'tokenization': 'word'}, {'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Tue Aug  8 08:34:56 2023", 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-huggingface': {'skip': False, 'vectorizePropertyName': False}}, 'name'

In [10]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-huggingface",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-huggingface": {
            "model": "sentence-transformers/all-MiniLM-L6-v2",  # Can be any public or private Hugging Face model.
            "options": {
                "waitForModel": True
            }
        }
    }
}

client.schema.create_class(class_obj)

In [11]:
# Load data
import requests
url = 'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Configure a batch process
with client.batch(
    batch_size=100
) as batch:
    # Batch import all Questions
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        client.batch.add_data_object(
            properties,
            "Question",
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [12]:
# Load data
import requests
fname = "jeopardy_tiny_with_vectors_all-MiniLM-L6-v2.json"  # This file includes vectors, created using `all-MiniLM-L6-v2`
url = f'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/{fname}'
resp = requests.get(url)
data = json.loads(resp.text)

# Configure a batch process
with client.batch(
    batch_size=100
) as batch:
    # Batch import all Questions
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        custom_vector = d["vector"]
        client.batch.add_data_object(
            properties,
            "Question",
            vector=custom_vector  # Add custom vector
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [17]:
schema = client.schema.get()
print(schema)

{'classes': [{'class': 'Question', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2', 'options': {'waitForModel': True}, 'vectorizeClassName': True}}, 'multiTenancyConfig': {'enabled': False}, 'properties': [{'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Tue Aug  8 08:34:56 2023", 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-huggingface': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'question', 'tokenization': 'word'}, {'dataType': ['text'], 'description': "This property was generated by Weaviate's auto-schema feature on Tue Aug  8 08:34:56 2023", 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-huggingface': {'skip': False, 'vectorizePropertyName': False}}, 'name'

In [13]:
nearText = {"concepts": ["biology"]}

response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text(nearText)
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "answer": "DNA",
                    "category": "SCIENCE",
                    "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
                },
                {
                    "answer": "DNA",
                    "category": "SCIENCE",
                    "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
                }
            ]
        }
    }
}


In [14]:
nearText = {"concepts": ["biology"]}

response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text(nearText)
    .with_where({
        "path": ["category"],
        "operator": "Equal",
        "valueText": "ANIMALS"
    })
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "answer": "Elephant",
                    "category": "ANIMALS",
                    "question": "It's the only living mammal in the order Proboseidea"
                },
                {
                    "answer": "the nose or snout",
                    "category": "ANIMALS",
                    "question": "The gavial looks very much like a crocodile except for this bodily feature"
                }
            ]
        }
    }
}
